# Kili Tutorial: Webhooks

In this tutorial, we will show how to use webhooks to monitor actions in Kili, such as a label creation. The goal of this tutorial is to illustrate some basic components and concepts of Kili in a simple way, but also to dive into the actual process of iteratively developing real applications in Kili.

Additionally:

For an overview of Kili, visit kili-technology.com You can also check out the Kili [documentation](https://cloud.kili-technology.com/docs). Our goal is to export labels that can predict whether an image contains a Porsche or a Tesla.

The tutorial is divided into two parts:

1. Why use webhooks?
2. Using Kili's webhook in Python

## 1. Why use webhooks?

Webhooks allow to react to particular action in Kili's database by triggering a callback whenever an action is completed. For instance, here, every time a label is created in frontend (upper panel), the label can be logged in Python (lower right panel):

![gif](./img/websockets.gif)

## 2. Using Kili's webhook in Python

Kili Playground exposes a method `label_created_or_updated` that allows to listen for all actions on labels:

- creation of a new label
- update of an existing label

First of all, you need to authenticate:

In [4]:
import os
import time

from kili.client import Kili


api_key = os.getenv('KILI_USER_API_KEY')
api_endpoint = os.getenv('KILI_API_ENDPOINT')

kili = Kili(api_key=api_key, api_endpoint=api_endpoint)

Then you can define a callback that will be triggered each time a label gets created/updated:

In [5]:
# Use create_project = False if you want to test it for your own project. create_project = True
# is when you want an end to end demo.
create_project = True
if create_project:
    json_interface = {
      "jobs": {
        "JOB_0": {
          "mlTask": "CLASSIFICATION",
          "required": 1,
          "content": {
            "categories": {
              "OBJECT_A": {
                "name": "Object A",
                "children": []
              },
              "OBJECT_B": {
                "name": "Object B",
                "children": []
              }
            },
            "input": "radio"
          }
        }
      }
    }
    project_id = kili.create_project(input_type='IMAGE', json_interface=json_interface, title='Webhook project')['id']
    kili.append_many_to_dataset(
        project_id=project_id,
        content_array=['https://storage.googleapis.com/label-public-staging/car/car_1.jpg'],
        external_id_array=['car_1']
    )
    asset_id = kili.assets(project_id=project_id, fields=['id'])[0]['id']
else:
    project_id = 'CHANGE_ME_FOR_YOUR_PROJECT_ID'

def callback(id, data):
    print(f'New data: {data}\n')

client = kili.label_created_or_updated(
    project_id=project_id, callback=callback)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 13.86it/s]


The method `label_created_or_updated` returns a client allowing you to manage the webhook. 

In [6]:
response = {
    'JOB_0': {
        'categories': [{'name': 'OBJECT_A'}]
    }
}
def add_label():
    kili.append_to_labels(
        json_response=response,
        label_asset_id=asset_id,
        label_type='INFERENCE'
    )

if create_project:
    add_label()

New data: {'type': 'data', 'id': 'lLSMLJ', 'payload': {'data': {'data': {'id': 'ckwxo4poz00b8al9k5qii7qih', 'jsonResponse': '{"JOB_0":{"categories":[{"name":"OBJECT_A"}]}}'}}}}



For example, you can pause or unpause the webhook : 

In [7]:
client.pause()
add_label()

In [8]:
client.unpause()
add_label()

New data: {'type': 'data', 'id': 'lLSMLJ', 'payload': {'data': {'data': {'id': 'ckwxo4r0b00bkal9kce8n7z2w', 'jsonResponse': '{"JOB_0":{"categories":[{"name":"OBJECT_A"}]}}'}}}}



Webhooks have a timeout of around 30 days. After this period, the hook is automatically killed. If you need to stop it before, you can call `close`:

In [9]:
client.close()
add_label()

Exception in thread Thread-9:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/kili/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/opt/anaconda3/envs/kili/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/maximeduval/Documents/kili-playground/kili/graphql_client.py", line 253, in subs
    response = json.loads(self._conn.recv())
  File "/opt/anaconda3/envs/kili/lib/python3.8/json/__init__.py", line 357, in loads
    return _default_decoder.decode(s)
  File "/opt/anaconda3/envs/kili/lib/python3.8/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/opt/anaconda3/envs/kili/lib/python3.8/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)


If you need to reset the hook, you can call `reset_timeout`:

In [10]:
client.reset_timeout()

08/12/2021 16:10:37 reconnected


Finally, you can get the number of seconds the webhook was up with :

In [11]:
time.sleep(3)
lifetime = client.get_lifetime()
print(f'Lifetime: {lifetime}')
assert lifetime > 0
assert len(kili.labels(project_id=project_id)) == 4

Lifetime: 2


## Summary
In this tutorial, we accomplished the following:

We introduced the concept of webhook and we used `label_created_or_updated` to trigger a webhook.

You can also visit the Kili [website](https://kili-technology.com/) or Kili [documentation](https://cloud.kili-technology.com/docs/introduction/introduction-to-kili-technology.html) for more info!